# Амангелді Нұрғалым - Зертханалық жұмыс, СИБ 23-01

## Импортируем необходимые библиотеки для создания генеративно-состязательной сети

In [1]:
# Импортируем необходимые библиотеки
import time  # Для работы с временем
import torch  # Основная библиотека PyTorch
import torch.nn as nn  # Модуль нейронных сетей PyTorch
import torch.optim as optim  # Оптимизаторы PyTorch
from torch.utils.data import DataLoader  # Загрузчик данных PyTorch
from torchvision import datasets  # Датасеты для компьютерного зрения PyTorch
from torchvision.transforms import transforms  # Преобразования изображений PyTorch
#from model import discriminator, generator #
import numpy as np  # Библиотека для работы с массивами данных
import matplotlib.pyplot as plt  # Библиотека для визуализации данных

## Определяем, доступны ли какие-либо графические процессоры

In [2]:
# Определяем устройство, на котором будет выполняться вычисления (GPU, если доступен, иначе CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Сетевые архитектуры
## Ниже приведены архитектуры дискриминатора и генератора

In [3]:
# Определение класса дискриминатора
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        # Первый полносвязный слой с 784 входами и 512 выходами
        self.fc1 = nn.Linear(784, 512)
        # Второй полносвязный слой с 512 входами и 1 выходом
        self.fc2 = nn.Linear(512, 1)
        # Функция активации LeakyReLU с коэффициентом наклона 0.1
        self.activation = nn.LeakyReLU(0.1)

    # Метод forward, определяющий проход вперед через сеть
    def forward(self, x):
        # При изменении формы тензора учтем размер батча и количество признаков
        x = x.view(-1, 784)
        # Применение активации к первому слою
        x = self.activation(self.fc1(x))
        # Применение второго слоя
        x = self.fc2(x)
        # Применение сигмоидной функции для получения вероятности
        return nn.Sigmoid()(x)

# Определение класса генератора
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        # Первый полносвязный слой с 128 входами и 1024 выходами
        self.fc1 = nn.Linear(128, 1024)
        # Второй полносвязный слой с 1024 входами и 2048 выходами
        self.fc2 = nn.Linear(1024, 2048)
        # Третий полносвязный слой с 2048 входами и 784 выходами
        self.fc3 = nn.Linear(2048, 784)
        # Функция активации ReLU
        self.activation = nn.ReLU()

    # Метод forward, определяющий проход вперед через сеть
    def forward(self, x):
        # Применение активации к первому слою
        x = self.activation(self.fc1(x))
        # Применение активации ко второму слою
        x = self.activation(self.fc2(x))
        # Применение третьего слоя
        x = self.fc3(x)
        # Изменение формы тензора, чтобы получить изображение 28x28
        x = x.view(-1, 1, 28, 28)
        # Применение гиперболического тангенса к выходу
        return nn.Tanh()(x)

In [4]:
# Определение функции потерь для дискриминатора
# Используется бинарная кросс-энтропия (BCELoss)
def d_loss_function(inputs, targets):
    return nn.BCELoss()(inputs, targets)

# Определение функции потерь для генератора
# Целевые метки для генератора - единицы (ожидаем, чтобы дискриминатор принимал сгенерированные изображения)
def g_loss_function(inputs):
    # Создание тензора целевых меток состоящего из единиц
    targets = torch.ones([inputs.shape[0], 1])
    targets = targets.to(device)
    # Используется бинарная кросс-энтропия (BCELoss)
    return nn.BCELoss()(inputs, targets)

## Определяем, доступны ли какие-либо графические процессоры

In [11]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

# Model
G = generator().to(device)
D = discriminator().to(device)
print(G)
print(D)

# Settings
epochs = 10
lr = 0.0002
batch_size = 64
g_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))


# Transform
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])


# Load data
train_set = datasets.MNIST('mnist/', train=True, download=True, transform=transform)
test_set = datasets.MNIST('mnist/', train=False, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

GPU State: cpu
generator(
  (fc1): Linear(in_features=128, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=784, bias=True)
  (activation): ReLU()
)
discriminator(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
  (activation): LeakyReLU(negative_slope=0.1)
)


### Процедура обучения сети.
### Этот код представляет собой цикл обучения для генеративно-состязательной сети (GAN). Дискриминатор и генератор обучаются поочередно, а затем сохраняется модель генератора после каждых 10 эпох.

In [12]:
for epoch in range(epochs):
    for idx, (imgs, _) in enumerate(train_loader):
        idx += 1
        # Обучение дискриминатора
        # real_inputs - изображения из набора данных MNIST
        # fake_inputs - изображения от генератора
        # real_inputs должны быть классифицированы как 1, а fake_inputs - как 0
        real_inputs = imgs.to(device)
        real_outputs = D(real_inputs)
        real_label = torch.ones(real_inputs.shape[0], 1).to(device)
        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_label = torch.zeros(fake_inputs.shape[0], 1).to(device)
        outputs = torch.cat((real_outputs, fake_outputs), 0)
        targets = torch.cat((real_label, fake_label), 0)
        d_loss = d_loss_function(outputs, targets)
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Обучение генератора
        # Цель генератора получить от дискриминатора 1 по всем изображениям
        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_targets = torch.ones([fake_inputs.shape[0], 1]).to(device)
        g_loss = g_loss_function(fake_outputs)
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        # Вывод информации о потерях на каждой итерации
        if idx % 100 == 0 or idx == len(train_loader):
            print('Epoch {} Iteration {}: discriminator_loss {:.3f} generator_loss {:.3f}'.format(epoch, idx, d_loss.item(), g_loss.item()))
    
    # Сохранение модели генератора каждые 10 эпох
    if (epoch+1) % 10 == 0:
        torch.save(G, 'Generator_epoch_{}.pth'.format(epoch))

Epoch 0 Iteration 100: discriminator_loss 0.595 generator_loss 0.987
Epoch 0 Iteration 200: discriminator_loss 0.636 generator_loss 0.775
Epoch 0 Iteration 300: discriminator_loss 0.607 generator_loss 0.826
Epoch 0 Iteration 400: discriminator_loss 0.660 generator_loss 0.806
Epoch 0 Iteration 500: discriminator_loss 0.630 generator_loss 0.798
Epoch 0 Iteration 600: discriminator_loss 0.571 generator_loss 0.814
Epoch 0 Iteration 700: discriminator_loss 0.482 generator_loss 0.923
Epoch 0 Iteration 800: discriminator_loss 0.482 generator_loss 0.925
Epoch 0 Iteration 900: discriminator_loss 0.460 generator_loss 1.143
Epoch 0 Iteration 938: discriminator_loss 0.486 generator_loss 1.135
Epoch 1 Iteration 100: discriminator_loss 0.485 generator_loss 1.209
Epoch 1 Iteration 200: discriminator_loss 0.466 generator_loss 1.248
Epoch 1 Iteration 300: discriminator_loss 0.398 generator_loss 1.589
Epoch 1 Iteration 400: discriminator_loss 0.402 generator_loss 1.434
Epoch 1 Iteration 500: discriminat

In [13]:
print('Model saved.')

Model saved.


### После 10 эпох мы можем построить наборы данных и увидеть результаты — цифры сгенерированные из случайных шумов: